<a href="https://colab.research.google.com/github/Yphy/NLP/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json
import time

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# df = pd.read_excel('./df.xlsx')
# new_df = df.copy()
# new_df['rmk'] = new_df['rmk'].fillna('미확인') #rmk에 NAN값이 있는 걸 치환
# new_df = new_df.dropna() #결측값 하나 있는것 제거
# new_df = new_df[new_df['rmk'] !='정연'] #변경사항이 있는 사업보고서 '정연' 제거(약 300개정도)
# okt = Okt() #Twitter(Okt) 형태소분석기 실행
# strs = new_df['str'] 
# events = [okt.morphs(x) for x in strs] #형태소분석 토큰화 실행
# kl = [] #리스트안에 형태소분석된 토큰들이 들어있는 형태라서 str로 바꿔서 새 컬럼에 추가해준다
# for k in events:
#     kl.append(' '.join(k))
# new_df['str_tokens'] = kl 으로 토큰화시킨 csv 파일이 df 파일


In [0]:
df = pd.read_csv('gdrive/My Drive/colab/dart/new_df.csv') #kospi200 기업 데이터 불러오기)
df_light = df.loc[:,('crp_nm','str','str_tokens')]


In [0]:
rcp_ls = df['rcp_no']  #연도 column 생성
rcp_ls = list(map(str,rcp_ls))
rcp__ls = []
for i in rcp_ls:
    rcp__ls.append(int(i[0:4])-1)
df_light.insert(0,'연도',rcp__ls) 

In [0]:
df_2018 = df_light[df_light['연도'] == 2018] #연도별 슬라이싱
df_2017 = df_light[df_light['연도'] == 2017]

In [0]:
ls = []
for i in df_light.연도:
    ls.append(str(i))
title_ls = ls+df_light.crp_nm

df_cosine =  df_light.copy()
df_cosine.insert(0,'title',title_ls)  #새로운 title column 생성

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_cosine['str_tokens'])
# 형태소 분석마친 토큰들에 대해서 tf-idf 수행
print(tfidf_matrix.shape) #토큰화를 했을 떄와 안했을때(str) 비교했을 때 단어 수가 약 3배 적어졌다

(1345, 46840)


In [19]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df_cosine.index, index=df_cosine['title']).drop_duplicates() #title에 대하여 index생성
print(indices.head())

title
2018삼성전자    0
2017삼성전자    1
2016삼성전자    2
2015삼성전자    3
2014삼성전자    4
dtype: int64


In [0]:
def get_similar_documents(title, cosine_sim =  cosine_sim):
    #선택한 문서의 title로 해당되는 index를 가져온다.
    idx = indices[title] 
    
    #모든 문서에 대해서 해당 문서와의 유사도를 구한다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도에 따라 문서 정렬
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    
    #가장 유사한 10개 문서 가져온다.
    sim_scores = sim_scores[1:11]
    
    #가장 유사한 10개 문서의 인덱스 가져온다
    doc_indices = [i[0] for i in sim_scores]
    
    #가장 유사한 10개 문서의 title을 리턴
    for i in sim_scores:
        print(df_cosine['title'].iloc[i[0]]+ '  ('+ "%0.2f" %(i[1])+ ')') 

In [0]:
def get_different_documents(title, cosine_sim =  cosine_sim):
    #선택한 문서의 title로 해당되는 index를 가져온다.
    idx = indices[title] 
    
    #모든 문서에 대해서 해당 문서와의 유사도를 구한다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도에 따라 문서 정렬
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    
    #가장 유사한 10개 문서 가져온다.
    sim_scores = sim_scores[-10:]
    
    #가장 유사한 10개 문서의 인덱스 가져온다
    doc_indices = [i[0] for i in sim_scores]
    
    #가장 유사한 10개 문서의 title을 리턴
    for i in sim_scores:
        print(df_cosine['title'].iloc[i[0]]+ '  ('+ "%0.4f" %(i[1])+ ')') 

In [22]:
get_similar_documents('2018삼성전자')

2017삼성전자  (0.78)
2015삼성전자  (0.70)
2016삼성전자  (0.70)
2014삼성전자  (0.68)
2013삼성전자  (0.64)
2012삼성전자  (0.61)
2018LG전자  (0.59)
2011삼성전자  (0.58)
2018LG이노텍  (0.57)
2010삼성전자  (0.53)


In [23]:
get_different_documents('2018삼성전자')

2014GKL  (0.1482)
2014GKL  (0.1482)
2010현대위아  (0.1430)
2015GKL  (0.1411)
2018GKL  (0.1391)
2010대웅  (0.1390)
2017GKL  (0.1384)
2009오뚜기  (0.1150)
2010오뚜기  (0.1141)
2010녹십자홀딩스  (0.1070)


In [24]:
get_different_documents('2017삼성바이오로직스')

2014GKL  (0.0900)
2015GKL  (0.0884)
2015코오롱인더  (0.0866)
2013코오롱인더  (0.0865)
2018GKL  (0.0848)
2012코오롱인더  (0.0828)
2014미래에셋대우  (0.0827)
2010녹십자홀딩스  (0.0740)
2010오뚜기  (0.0727)
2009오뚜기  (0.0697)


In [26]:
tfidfv = TfidfVectorizer().fit(df_cosine['str_tokens'])
print(tfidfv.transform(df_cosine['str_tokens']).toarray())
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'이사': 40626, '경영': 29646, '진단': 43236, '분석': 35556, '의견': 40315, '예측': 39313, '정보': 42210, '대한': 32245, '주의': 42723, '사항': 36146, '자료': 41477, '미래': 34288, '포함': 45300, '하고': 45547, '있습니다': 41386, '이는': 40440, '과거': 30031, '아닌': 38095, '사건': 36004, '관계': 30073, '으로': 40218, '회사': 46656, '향후': 46185, '예상': 39293, '되는': 32632, '현황': 46273, '재무': 41760, '실적': 37930, '의미': 40331, '표현': 45346, '으로는': 40223, '전망': 42010, '계획': 29768, '기대': 30696, '같은': 29249, '단어': 31944, '합니다': 45884, '성격': 36726, '불확실한': 35742, '언급': 38685, '하는데': 45589, '긍정': 30661, '또는': 33027, '부정': 35425, '영향': 39268, '미칠': 34387, '있는': 41365, '불확실': 35740, '에는': 38828, '다음': 31849, '됩니다': 32740, '환율': 46558, '이자율': 40758, '변동': 35009, '국내외': 30288, '금융': 30588, '시장': 37650, '동향': 32601, '사업': 36080, '처분': 43557, '매수': 33736, '전략': 41999, '의사결정': 40333, '반도체': 3458

In [27]:
tfd = pd.DataFrame(tfidfv.vocabulary_.items(), columns=['단어' , '빈도'])
tfd.sort_values('빈도',ascending=False)

,단어,빈도
14633,힙입어,46839
16757,힘찬,46838
7532,힘차게,46837
35113,힘입,46836
18172,힘쓸,46835
...,...,...
21554,000000000000,4
39627,00000000,3
45233,0000,2
30370,000,1
